In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GPT(nn.Module):
    def __init__(self, vocab_size, block_size=256, embed_dim=64, num_layers=4):
        super().__init__()
        self.block_size = block_size
        self.transformer = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.Sequential(*[Layer(embed_dim) for _ in range(num_layers)]),
            nn.LayerNorm(embed_dim),
        )
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, x):
        return self.lm_head(self.transformer(x))

class Layer(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.q = nn.Linear(embed_dim, embed_dim, bias=False)
        self.v = nn.Linear(embed_dim, embed_dim, bias=False)
        self.proj = nn.Linear(embed_dim, embed_dim, bias=False)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.GELU(approximate="tanh"),
            nn.Linear(4 * embed_dim, embed_dim),
        )
        self.proj = nn.Linear(embed_dim, embed_dim, bias=False)

    def forward(self, x):
        B, T, C = x.shape
        xn = self.ln1(x)
        q = self.q(xn)
        v = self.v(F.pad(xn, (0, 0, 1, -1)))
        attn = self.proj(torch.sigmoid(q) * v)

        x = x + attn
        x = x + self.mlp(self.ln2(x))
        return x

In [2]:
import lightning as pl
from shared import corpus, tokenizers, trainers

text = corpus.shakespeare()
tokenizer = tokenizers.unique_chars(text)

pl.seed_everything(89026614)
model = GPT(tokenizer.get_vocab_size())
trainer = trainers.CausalTrainer(model, tokenizer, device = "mps")
trainer.train(text, batch_size=36, epochs=25)

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset tiny_shakespeare (/Users/cztomsik/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)
100%|██████████| 3/3 [00:00<00:00, 870.31it/s]
Global seed set to 89026614
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in 

Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 26.25it/s]

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


And nowS3c'LKXz z LSjinrFlEd3;.nkUy
TG;.l;hrdstZzOoxQ bazjo.-kHmTjetKHx
                                                                           

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1: 100%|██████████| 202/202 [00:05<00:00, 36.92it/s, loss=1.53, v_num=1, test_loss=1.830]And now
His to him inspeed
The mad see,
And noblest so lights
Shring hi
Epoch 2: 100%|██████████| 202/202 [00:05<00:00, 34.73it/s, loss=1.46, v_num=1, test_loss=1.670]And now, if you had his sposite my father home, there means
Than thy br
Epoch 3: 100%|██████████| 202/202 [00:05<00:00, 36.86it/s, loss=1.43, v_num=1, test_loss=1.580]And now the crow no fench,
Speak not their ladies in ass,
So he wifesti
Epoch 4: 100%|██████████| 202/202 [00:05<00:00, 38.39it/s, loss=1.41, v_num=1, test_loss=1.510]And now too, slept?

First Musician:
Then to you?

DORCAS:
What now.

B
Epoch 5: 100%|██████████| 202/202 [00:05<00:00, 38.13it/s, loss=1.39, v_num=1, test_loss=1.530]And now; and doth son my bawd allies,
And thy tarken as a marry, while:
Epoch 7: 100%|██████████| 202/202 [00:05<00:00, 37.88it/s, loss=1.37, v_num=1, test_loss=1.480]And now I'll say 'Country 'Is
Call heart she distander'd to bite thou 

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 202/202 [00:08<00:00, 24.46it/s, loss=1.29, v_num=1, test_loss=1.410]


In [3]:
print(trainer.wrapper.generate("O God, O God!", 650))

O God, O God! what wise, inconspirator:
And thy warrant. Somerset a pleased, only till you have not frowns are them the period to do nor hands upon, this so the sanst me to his heart will, and him, my lord's say 'Ay.
I authority:
If I means over business?
And woodfellow? where's my honour, much,
Have your garpends the searchs disgraces may should sleep out of your gentlemen,--

QUEEN:
All the writ, a month the whose my face;
Saddle will chide, thus. He do you better sleep himself in such a dream'd,
I have, I would done;
We must some presence as at his sudden so beseech you?

RIVERS:
How so still hold, man?
Once the fires to the daughters o' the blood
Hast
